In [277]:
import sqlite3
conn = sqlite3.connect('manufacturer.sqlite')
cur = conn.cursor()
#Task1 
# Writing a one formula in sql where you examine that the table is exists and if yes then remove it.


cur.execute('DROP TABLE IF EXISTS Products;')
cur.execute('DROP TABLE IF EXISTS Suppliers;')
cur.execute('DROP TABLE IF EXISTS Components;')
cur.execute('DROP TABLE IF EXISTS Inventory;')

#Create Tables
cur.execute('CREATE TABLE IF NOT EXISTS Suppliers(Id INTEGER PRIMARY KEY ,Name TEXT);')
cur.execute('CREATE TABLE IF NOT EXISTS Components(Id INTEGER PRIMARY KEY ,Name TEXT);')
cur.execute('''CREATE TABLE IF NOT EXISTS Products(Id INTEGER PRIMARY KEY ,Date TEXT ,Demand INTEGER ,Component_id INTEGER,Quantity INTEGER,Supplier_id INTEGER,
               FOREIGN KEY (Supplier_id) REFERENCES Suppliers(Id),
               FOREIGN KEY (Component_id) REFERENCES Components(Id));''')
cur.execute('''CREATE TABLE IF NOT EXISTS Inventory(Id INTEGER PRIMARY KEY ,Production_Backlog INTEGER,Stock_at_Hand INTEGER,Component_id INTEGER,
               FOREIGN KEY (Component_id) REFERENCES Components(Id));''')




 

In [288]:
class Manufactury:
    
    def add_components(self,c_name):
        conn = sqlite3.connect('manufacturer.sqlite')
        cur = conn.cursor()
        cur.execute("SELECT Id FROM Components WHERE Name =  ? ",(c_name,))
# gets the number of rows affected by the command executed
        row = cur.fetchone()
        if row != None:
            return  row[0]
        elif row == None:
#print("There are no results for this query")
            cur.execute("INSERT INTO Components(Name) VALUES(?);", (c_name,))
            conn.commit()
            self.add_components(c_name)

    def add_suppliers(self,s_name):
        conn = sqlite3.connect('manufacturer.sqlite')
        cur = conn.cursor()
        cur.execute("SELECT Id FROM Suppliers  WHERE Name =  ? ",(s_name,))
# gets the number of rows affected by the command executed
        row = cur.fetchone()
        if row != None:
            return  row[0]
        elif row == None:
#print("There are no results for this query")
            cur.execute("INSERT INTO Suppliers(Name) VALUES(?);", (s_name,))
            conn.commit()
            self.add_suppliers(s_name)
    
    def add_inventory(self,l_inv,comp):
        conn = sqlite3.connect('manufacturer.sqlite')
        cur = conn.cursor()
        cur.execute("INSERT INTO Inventory(Production_Backlog,Stock_at_Hand ,Component_id ) VALUES(?,?,?);", (l_inv[0],l_inv[1],self.add_components(comp),))
        conn.commit()
        
    def add_products(self,l_date,l_demand,l_sup,comp):
        for j in range(len(l_sup)):
            for i in range(len(l_date)):
                conn = sqlite3.connect('manufacturer.sqlite')
                cur = conn.cursor()
                cur.execute("INSERT INTO Products(Date,Demand ,Component_id,Quantity,Supplier_id  ) VALUES(?,?,?,?,?);", (l_date[i],l_demand[i],self.add_components(comp),l_sup[j][i],self.add_suppliers(l_sup[j][-1]),))
                conn.commit() 
    
#Task 2 Calculate the Material Balance 
    def calculate_Material_Balance(self,comp): 
        Quantity=0
        conn = sqlite3.connect('manufacturer.sqlite')
        cur = conn.cursor()
        cur.execute('''SELECT p.date,i.Stock_at_Hand - i.Production_Backlog-p.Demand,sum(p.Quantity) 
                            FROM Inventory i,Products p
                            WHERE i.Component_id=p.Component_id AND p.Component_id =  ? 
                            GROUP by p.date
                            ORDER by p.date
                            LIMIT 1''' ,(comp,))
        inventory = cur.fetchone()
        Material_Balance=inventory[1]
        Quantity=inventory[2]
        print('Date: '+inventory[0]+', Material_Balance:',Material_Balance)
        
        cur.execute("SELECT Date,Demand,sum(Quantity) FROM Products  WHERE Component_id =  ? AND Date not like ?  Group by Date",(comp,inventory[0],))
        row = cur.fetchall()
      
        for j in  row :
            Material_Balance=Material_Balance-j[1]+Quantity
            Quantity=j[2]
            print('Date: '+j[0]+', Material_Balance:',Material_Balance)
                              
#Task 3 How many confirmed delivery do we have in each supplier in all period? 
    def find_confirmed_deliveries_line(self):
            conn = sqlite3.connect('manufacturer.sqlite')
            cur = conn.cursor()
            cur.execute('''SELECT count(DISTINCT  p.Supplier_id) ,c.Name FROM Products p , Components c WHERE c.id=p.Component_id
                            GROUP by c.Name''') 
            result= cur.fetchall()
            print(result)

In [272]:
import pandas as pd
xl = pd.ExcelFile('2. Task.xlsx')
counter=0
c_date=[]
demand=[]
sup_l=[]
inv=[]
Manf=Manufactury()
for i in range(len(xl.sheet_names)):
    manufacturer= pd.read_excel('2. Task', sheet_name=xl.sheet_names[i],header=0)
    d=[i for i in manufacturer['TOTAL'][3:21].values.tolist()  if str(i)!= 'nan'] 
    Manf.add_components(xl.sheet_names[i])
    for n in range(len(d)+4):
        manufacturer= pd.read_excel('2. Task.xlsx', sheet_name=xl.sheet_names[i],header=n)
        if n==1:
            inv.append(manufacturer.columns[3:4][0])
            for k in range(len(manufacturer.columns[4:])):
                c_date.append(str(manufacturer.columns[4:][k]))          
        elif n==2:
            inv.append(manufacturer.columns[3:4][0])
        elif n==3:
            demand=([int(float(x)) for x in manufacturer.columns[4:]]) 
            #print(demand)
        elif n>=4:
            sup=([int(float(x)) for x in manufacturer.columns[4:]]) 
            sup.append(d[counter])
            sup_l.append(sup)
            Manf.add_suppliers(d[counter])
            counter=counter+1
    #['Smith Ltd', 'Common Ltd']        
    Manf.add_inventory(inv,xl.sheet_names[i])
    Manf.add_products(c_date,demand,sup_l,xl.sheet_names[i])
    Manf.calculate_Material_Balance(1) 


Date: 2021-12-13 00:00:00, Material_Balance: 67523
Date: 2021-12-20 00:00:00, Material_Balance: 66573
Date: 2021-12-27 00:00:00, Material_Balance: 65573
Date: 2022-01-03 00:00:00, Material_Balance: 64673
Date: 2022-01-10 00:00:00, Material_Balance: 55673
Date: 2022-01-17 00:00:00, Material_Balance: 115673
Date: 2022-01-24 00:00:00, Material_Balance: 115673
Date: 2022-01-31 00:00:00, Material_Balance: 105673
Date: 2022-02-07 00:00:00, Material_Balance: 40673
Date: 2022-02-14 00:00:00, Material_Balance: -29327
Date: 2022-02-21 00:00:00, Material_Balance: -21827
Date: 2022-02-28 00:00:00, Material_Balance: -21827
Date: 2022-03-07 00:00:00, Material_Balance: -31827
Date: 2022-03-14 00:00:00, Material_Balance: -31827
Date: 2022-03-21 00:00:00, Material_Balance: -31827
Date: 2022-03-28 00:00:00, Material_Balance: -33827
Date: 2022-04-04 00:00:00, Material_Balance: -11327
Date: 2022-04-11 00:00:00, Material_Balance: -11327
Date: 2022-04-18 00:00:00, Material_Balance: 46173
Date: 2022-04-25 00

In [278]:
import pandas as pd
xl = pd.ExcelFile('3.Task.xlsx')
sup_l=[]

Manf=Manufactury()
for i in range(len(xl.sheet_names)):
    sup_l=[]
    c_date=[]
    demand=[]
    inv=[]
    counter=0
    manufacturer= pd.read_excel('3.Task.xlsx', sheet_name=xl.sheet_names[i],header=0)
    d=[i for i in manufacturer['TOTAL'][3:21].values.tolist()  if str(i)!= 'nan'] 
    Manf.add_components(xl.sheet_names[i])
    for n in range(len(d)+4):
        manufacturer= pd.read_excel('3.Task.xlsx', sheet_name=xl.sheet_names[i],header=n)
        if n==1:
            inv.append(manufacturer.columns[3:4][0])
            for k in range(len(manufacturer.columns[4:])):
                c_date.append(str(manufacturer.columns[4:][k]))          
        elif n==2:
            inv.append(manufacturer.columns[3:4][0])
        elif n==3:
            demand=([int(float(x)) for x in manufacturer.columns[4:]]) 
            #print(demand)
        elif n>=4:
            sup=([int(float(x)) for x in manufacturer.columns[4:]]) 
            sup.append(d[counter])
            sup_l.append(sup)
            Manf.add_suppliers(d[counter])
            counter=counter+1
    #['Smith Ltd', 'Common Ltd']        
    Manf.add_inventory(inv,xl.sheet_names[i])
    Manf.add_products(c_date,demand,sup_l,xl.sheet_names[i])



In [289]:
Manf=Manufactury()
Manf.find_confirmed_deliveries_line()

[(2, 'FG-AA123456'), (1, 'FG-BB654321'), (3, 'FG-CC987654')]
